# Raking worksheet

*Last updated: 9/23/2019 by Andrew Therriault*

This worksheet shows how a simple rake weighting algorithm can be used to balance an unrepresentative dataset and reduce bias in survey estimates.

In [1]:
import numpy as np
np.random.seed(1234)
import pandas as pd

## Creating an artificial dataset with random values of gender, race, and location for 500 respondents

In [2]:
df = pd.DataFrame()
n = 1000
df['gender'] = np.random.choice(['female','male'], n, p=[0.4,0.6])
df['race'] = np.random.choice(['black','white','other'], n, p=[0.1,0.7,0.2])
df['location'] = np.random.choice(['urban','suburban','rural'], n, p=[0.2,0.4,0.4])
df.sample(10)

,gender,race,location
881,male,white,urban
81,male,white,rural
933,female,white,rural
391,female,white,suburban
718,male,white,suburban
654,female,white,suburban
463,male,white,rural
445,female,white,suburban
657,male,other,urban
228,female,other,rural


## Setting the targets for those characteristics (as marginal distributions) in the broader population
These numbers are such that the dataframe is definitely not representative. They are generated independently here, though in the real world they'll probably be correlated.

In [3]:
targets = {
    'gender': {'female': 0.5, 'male': 0.5},
    'race': {'black': 0.2, 'white': 0.5, 'other': 0.3},
    'location': {'urban': 0.4, 'suburban': 0.35, 'rural': 0.25} 
}

## Defining raking function
This function takes three inputs:
* a dataset formatted as a dataframe with named columns (in the example, 'gender', 'race', and 'location')
* a set of target marginal distributions for each of the variables to be weighted upon, as a dictionary with the first key as the column name in the dataset and the second key as the column value in the dataset, with the value of that second key being the desired proportion
* the number of iterations to run for (start small and then observe if convergence occurs)

The algorithm I use here is very simple and doesn't account for things like trimming of extreme weights, but will work for this purpose (look at actual weighting packages for more detailed versions).

Also, note that after every iteration, I store the weights on each variable that apply to each individual in the main dataframe and recalculate those as a series of successive column multiplications. There's probably a more efficient way to do this which doesn't require setting all those column vaues and instead just applies weights based on characteristics all at once (and that's what I did in an earlier version of this using just two variables), but figuring out how to do that join for an arbitrary number of variables included in weighting makes it a lot more complicated. So if someone wants to submit a pull request for a better version, go for it, but this works fine and any CPU / memory inefficiency is negligible at this scale.

In [4]:
def rake(data, m=targets, iterations=1):
    
    data['w'] = 1 #initializing individual weights to 1
    mstar = dict() #creating dictionary of marginal distributions of the data
    weights = dict() #creating dictionary of weights for each variable and value
    
    #looping over variables to show initial marginal distributions, initialize individual per-variable weights,
    #and initialize variable-value weights
    for variable in targets:
        print('Initial marginal distributions')
        print(data.groupby(variable).w.sum()/len(data))
        data['w_{}'.format(variable)] = 1 #initializing individual weights per variable (to be combined later)
        weights[variable] = dict()
        for value in targets[variable]:
            weights[variable][value] = 1 #initializing variable-value weights to 1
    
    #looping over number of iterations specified
    for i in range(iterations):
        print('\nIteration {}'.format(i+1))
        #looping over variables to update weights for one variable at a time
        for variable in targets:
            mstar[variable] = dict()
            #loopiing over values to update weights for each variable / value combination
            for value in targets[variable]:
                mstar[variable][value] = data[data[variable]==value].w.sum()/len(data) #updating marginal distribution
                weights[variable][value] = weights[variable][value] * m[variable][value] / mstar[variable][value] #updating weight
                data.loc[data[variable]==value,'w_{}'.format(variable)] = weights[variable][value] #setting weight individual weight for variable
            #recalculating individual weights using all per-variable weights
            data['w'] = 1
            for variable2 in targets: #looping over variables within this to update weights across all
                data['w'] = data['w'] * data['w_{}'.format(variable2)]
        #showing weights after iteration            
        print('Weights:')
        print(pd.DataFrame(weights).unstack().dropna())
    print('\nFinal marginal distributions:')
    for variable in targets:
        print(data.groupby(variable).w.sum()/len(data))

## Trying out the raking function

In [5]:
rake(df, iterations=5)

Initial marginal distributions
gender
female    0.385
male      0.615
Name: w, dtype: float64
Initial marginal distributions
race
black    0.130
other    0.195
white    0.675
Name: w, dtype: float64
Initial marginal distributions
location
rural       0.412
suburban    0.382
urban       0.206
Name: w, dtype: float64

Iteration 1
Weights:
gender    female      1.298701
          male        0.813008
race      black       1.544521
          other       1.516088
          white       0.743348
location  rural       0.607510
          suburban    0.897418
          urban       2.015351
dtype: float64

Iteration 2
Weights:
gender    female      1.333408
          male        0.792383
race      black       1.437017
          other       1.622402
          white       0.736431
location  rural       0.602297
          suburban    0.893687
          urban       2.033783
dtype: float64

Iteration 3
Weights:
gender    female      1.333014
          male        0.792618
race      black       1.43532

##### So by the 5th iteration, the weights have converged to a stable set of values, and the weighted marginal distributions match the targets exactly.

## Creating a synthetic outcome to test out
This is a really simple binary outcome that has a probability of 0.5 of being = 1 for black females and urban males, and is always 0 for everyone else. Given that the distributions of demographics are independent, black females repesent 10% of the population and urban males represent 20%, so with 30% of the population between them and half of those peoples' responses being = 1, the actual population distribution of this outcome is 15% = 1, 85% = 0. Let's see how close we get with weighted and unweighted estimates.

In [6]:
df['x'] = 0 #starting with 0, then updating for the two chosen groups
df.loc[((df.gender=='female')&(df.race=='black')),'x'] = np.random.binomial(1, 0.5, size=
                                                    len(df[((df.gender=='female')&(df.race=='black'))]))
df.loc[((df.gender=='male')&(df.location=='urban')),'x'] = np.random.binomial(1, 0.5, size=
                                                    len(df[((df.gender=='male')&(df.location=='urban'))]))

#### Unweighted analysis

In [7]:
mean = df.x.mean()
sem = df.x.sem()
print('Mean   = {}'.format(mean))
print('SE     = {}'.format(sem))
print('95% CI = ({},{})'.format(mean-1.96*sem,mean+1.96*sem))

Mean   = 0.095
SE     = 0.009276910103103306
95% CI = (0.07681725619791752,0.11318274380208249)


#### Weighted analysis (using statsmodels for weighting)

In [8]:
from statsmodels.stats.weightstats import DescrStatsW
mean = DescrStatsW(df.x, df.w).mean
sem = DescrStatsW(df.x, df.w).std_mean
print('Mean   = {}'.format(mean))
print('SE     = {}'.format(sem))
print('95% CI = ({},{})'.format(mean-1.96*sem,mean+1.96*sem))


Mean   = 0.15924838744083486
SE     = 0.011576802706240952
95% CI = (0.1365578541366026,0.18193892074506712)


##### So the takeaway here is that the unweighted analysis provided a biased estimate, while the weighted analysis got substantially closer to the actual value of 0.15. And the true result was within the 95% confidence interval, though that's not guaranteed. (You can try setting different sample sizes and random seeds at the beginning and see how these results change - as sample sizes increase, the weighted result will converge on the true value, but the unweighted result will not.)